In [1]:
##mounting drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
def detect_face(image_path, img_size=(92, 112), scaleFactor=1.1, minNeighbors=5):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=minNeighbors)

    if len(faces) == 0:
        # print(f"No face detected in {image_path}")
        return None

    # Assume largest face is the correct one
    x, y, w, h = max(faces, key=lambda f: f[2]*f[3])
    face = gray[y:y+h, x:x+w]
    face_resized = cv2.resize(face, img_size)

    return face_resized


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict

def extract_subject_id(filename):
    return filename.split('-')[0]

def load_dataset(train_dir, test_dir, image_size=(100, 100)):
    X_train, y_train = [], []
    X_test, y_test = [], []
    x_test_paths = []

    label_map = {}
    label_counter = 0

    def process_folder(folder, is_train=True):
        nonlocal label_counter
        X, y, paths = [], [], []

        for fname in sorted(os.listdir(folder)):
            if not fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                continue

            subject_id = extract_subject_id(fname)
            if subject_id not in label_map:
                label_map[subject_id] = label_counter
                label_counter += 1
            label = label_map[subject_id]

            img_path = os.path.join(folder, fname)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Failed to read {img_path}")
                continue

            face = detect_face(img_path)
            if face is None:
                # print(f"No face found in {img_path}")
                continue

            face_resized = cv2.resize(face, image_size)
            X.append(face_resized)
            y.append(label)
            paths.append(img_path)

        return X, y, paths

    X_train, y_train, _ = process_folder(train_dir, is_train=True)
    X_test, y_test, x_test_paths = process_folder(test_dir, is_train=False)

    print(f"Loaded {len(X_train)} training faces.")
    print(f"Loaded {len(X_test)} testing faces.")
    return (np.array(X_train), np.array(y_train),
            np.array(X_test), np.array(y_test),
            label_map, x_test_paths)


In [5]:
dataset_path = "/content/drive/Shared with me/FEI_faces"
testing_path = "/content/drive/Shared with me/FEI_testing"
X_train, y_train, X_test, y_test, label_map, X_test_files = load_dataset(dataset_path, testing_path)

print(f"Train samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Subjects found: {len(label_map)}")
print(f"Example label map: {label_map}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shared with me/FEI_faces'

In [6]:
print(X_test_files)

NameError: name 'X_test_files' is not defined

In [7]:
## testing detection on test images bc they aren't cropped yet
image_path = X_test_files[17]  # Pick one test image
detected_face = detect_face(image_path)

if detected_face is not None:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.title(f"Original Image - {image_path}")
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(detected_face, cmap='gray')
    plt.title("Detected Face")
    plt.axis('off')
    plt.show()


NameError: name 'X_test_files' is not defined